In [1]:
import tensorflow as tf
from tensorflow.keras import layers
import utils
from tensorflow.keras import mixed_precision
import os
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
mixed_precision.set_global_policy('mixed_float16')
import logging
tf.get_logger().setLevel(logging.ERROR)

2024-07-29 15:18:10.672117: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-29 15:18:10.697247: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-29 15:18:10.697275: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-29 15:18:10.697298: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-29 15:18:10.703321: I tensorflow/core/platform/cpu_feature_g

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 4070 Laptop GPU, compute capability 8.9


2024-07-29 15:18:13.205279: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-29 15:18:13.228308: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-29 15:18:13.243337: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [2]:
"""
#Batching using prefetch
train_data_casted = train_data.map(map_func = preprocess_image, num_parallel_calls = tf.data.AUTOTUNE).shuffle(buffer_size = 1000).batch(batch_size = 32).prefetch(buffer_size = tf.data.AUTOTUNE)
test_data_casted = test_data.map(map_func = preprocess_image, num_parallel_calls = tf.data.AUTOTUNE).batch(32).prefetch(tf.data.AUTOTUNE)
"""

'\n#Batching using prefetch\ntrain_data_casted = train_data.map(map_func = preprocess_image, num_parallel_calls = tf.data.AUTOTUNE).shuffle(buffer_size = 1000).batch(batch_size = 32).prefetch(buffer_size = tf.data.AUTOTUNE)\ntest_data_casted = test_data.map(map_func = preprocess_image, num_parallel_calls = tf.data.AUTOTUNE).batch(32).prefetch(tf.data.AUTOTUNE)\n'

In [3]:
fundus_train = "/home/thefilthysalad/PycharmProjects/eye_detection_fundus_dataset/Dataset/split1/train"
fundus_test = "/home/thefilthysalad/PycharmProjects/eye_detection_fundus_dataset/Dataset/split1/test"


BATCH_SIZE = 32
IMG_SIZE = (224, 224)


In [4]:
print(os.listdir(fundus_train))

['glaucoma', 'normal', 'cataract', 'diabetic_retinopathy']


In [5]:
IMG_HEIGHT, IMG_WIDTH = 224, 224

In [6]:
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    fundus_train,
    labels='inferred',
    label_mode='categorical',
    batch_size=BATCH_SIZE,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    shuffle=True,
    seed=123,
    validation_split=0.3,
    subset='training'
)

validation_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    fundus_train,
    labels='inferred',
    label_mode='categorical',
    batch_size=BATCH_SIZE,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    shuffle=False,
    seed=123,
    validation_split=0.3,
    subset='validation'
)
test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    fundus_test,
    labels='inferred',
    label_mode='categorical',
    batch_size=BATCH_SIZE,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    shuffle=False,
    seed=123,

)

Found 3372 files belonging to 4 classes.
Using 2361 files for training.


2024-07-29 15:18:13.323339: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-29 15:18:13.326109: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-29 15:18:13.329470: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

Found 3372 files belonging to 4 classes.
Using 1011 files for validation.
Found 845 files belonging to 4 classes.


In [7]:
train_dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 4), dtype=tf.float32, name=None))>

In [8]:
#Using tf prefetch dataset
preprocess_input = tf.keras.applications.resnet.preprocess_input

In [9]:
train_datagen = train_dataset.map(lambda x, y: (preprocess_input(x), y))
val_datagen = validation_dataset.map(lambda x, y: (preprocess_input(x), y))
test_datagen = test_dataset.map(lambda x, y: (preprocess_input(x), y))

In [10]:
train_datagen

<_MapDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 4), dtype=tf.float32, name=None))>

In [11]:
import tensorflow.keras.applications as apps
IMG_WIDTH, IMG_HEIGHT = 224, 224
base_model = apps.ResNet101(weights = 'imagenet', include_top = False, input_shape = (IMG_WIDTH, IMG_HEIGHT, 3))
base_model.trainable = False

In [12]:

for i in base_model.layers:
    print(f'Layer: {i.name}, {i.trainable}')

Layer: input_1, False
Layer: conv1_pad, False
Layer: conv1_conv, False
Layer: conv1_bn, False
Layer: conv1_relu, False
Layer: pool1_pad, False
Layer: pool1_pool, False
Layer: conv2_block1_1_conv, False
Layer: conv2_block1_1_bn, False
Layer: conv2_block1_1_relu, False
Layer: conv2_block1_2_conv, False
Layer: conv2_block1_2_bn, False
Layer: conv2_block1_2_relu, False
Layer: conv2_block1_0_conv, False
Layer: conv2_block1_3_conv, False
Layer: conv2_block1_0_bn, False
Layer: conv2_block1_3_bn, False
Layer: conv2_block1_add, False
Layer: conv2_block1_out, False
Layer: conv2_block2_1_conv, False
Layer: conv2_block2_1_bn, False
Layer: conv2_block2_1_relu, False
Layer: conv2_block2_2_conv, False
Layer: conv2_block2_2_bn, False
Layer: conv2_block2_2_relu, False
Layer: conv2_block2_3_conv, False
Layer: conv2_block2_3_bn, False
Layer: conv2_block2_add, False
Layer: conv2_block2_out, False
Layer: conv2_block3_1_conv, False
Layer: conv2_block3_1_bn, False
Layer: conv2_block3_1_relu, False
Layer: con

In [13]:
len(base_model.layers)

345

In [14]:
No_of_classes = len(os.listdir(fundus_train))
No_of_classes

4

In [15]:
aug_layer = utils.return_data_aug_layer_for_eff_net()

In [17]:
from keras.src.regularizers import l2
from tensorflow.keras.layers import Dense, GlobalAvgPool2D, Input, Dropout, BatchNormalization
from tensorflow.keras.models import Model
inputs = Input(shape = (IMG_HEIGHT, IMG_WIDTH, 3), name = 'Input_layer')
x = aug_layer(inputs)
#x = tf.keras.layers.Rescaling(1./255)(inputs)
x = base_model(x, training = False)
x = layers.GlobalAvgPool2D()(x)
x = BatchNormalization()(x)

x = Dense(512, kernel_regularizer=l2(0.01))(x)
x = BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)

x = Dense(256, kernel_regularizer=l2(0.01))(x)
x = BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
x = Dropout(0.3)(x)

x = Dense(128, kernel_regularizer=l2(0.01))(x)
x = BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
x = Dropout(0.3)(x)


x = Dense(64, kernel_regularizer=l2(0.01))(x)
x = BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
x = Dropout(0.3)(x)
Outputs = Dense(No_of_classes, activation = 'softmax', dtype = tf.float32)(x)

model_1 = Model(inputs, Outputs, name = 'Resnet_101')

model_1.summary()

Model: "Resnet_101"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 Data_Augmentation (Sequent  (None, None, None, 3)     0         
 ial)                                                            
                                                                 
 resnet101 (Functional)      (None, 7, 7, 2048)        42658176  
                                                                 
 global_average_pooling2d_1  (None, 2048)              0         
  (GlobalAveragePooling2D)                                       
                                                                 
 batch_normalization_5 (Bat  (None, 2048)              8192      
 chNormalization)                                                
                                                        

In [18]:
#Model checkpointing
from tensorflow.keras.callbacks import ModelCheckpoint
model_1chkpt = ModelCheckpoint(filepath = os.path.join('Trained_Models',model_1.name), save_weights_only = False, save_best_only = True, verbose = 1)

In [19]:
from keras.src.callbacks import ReduceLROnPlateau

lr_scheduler = ReduceLROnPlateau(factor=0.3, patience=3) 

In [20]:
#model compilation
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adam
model_1.compile(loss = 'categorical_crossentropy', optimizer = Adam(learning_rate = 0.001), metrics = ['accuracy'])

In [21]:
history_1 = model_1.fit(train_datagen, validation_data = (val_datagen), epochs = 20, verbose = 1, callbacks = [model_1chkpt, lr_scheduler], steps_per_epoch = len(train_datagen), validation_steps = len(val_datagen))

Epoch 1/20


2024-07-29 15:18:40.540701: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8600
2024-07-29 15:18:40.625690: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-07-29 15:18:41.858140: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x732e74003470 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-07-29 15:18:41.858179: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 4070 Laptop GPU, Compute Capability 8.9
2024-07-29 15:18:41.864962: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-07-29 15:18:41.937167: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


74/74 [==============================] - ETA: 0s - loss: 13.2548 - accuracy: 0.5909
Epoch 1: val_loss improved from inf to 11.26959, saving model to Trained_Models/Resnet_101
74/74 [==============================] - 53s 651ms/step - loss: 13.2548 - accuracy: 0.5909 - val_loss: 11.2696 - val_accuracy: 0.2502 - lr: 0.0010
Epoch 2/20
74/74 [==============================] - ETA: 0s - loss: 8.1726 - accuracy: 0.7501
Epoch 2: val_loss improved from 11.26959 to 6.44356, saving model to Trained_Models/Resnet_101
74/74 [==============================] - 38s 517ms/step - loss: 8.1726 - accuracy: 0.7501 - val_loss: 6.4436 - val_accuracy: 0.6518 - lr: 0.0010
Epoch 3/20
74/74 [==============================] - ETA: 0s - loss: 4.9559 - accuracy: 0.7726
Epoch 3: val_loss improved from 6.44356 to 4.37699, saving model to Trained_Models/Resnet_101
74/74 [==============================] - 37s 494ms/step - loss: 4.9559 - accuracy: 0.7726 - val_loss: 4.3770 - val_accuracy: 0.3848 - lr: 0.0010
Epoch 4/20


In [22]:
model_1.evaluate(test_datagen)

27/27 [==============================] - 2s 63ms/step - loss: 0.9564 - accuracy: 0.7065


[0.9563683867454529, 0.7065088748931885]

In [23]:
base_model.trainable = True

In [24]:
for layer in base_model.layers[:-25]:
    layer.trainable = False

In [25]:
for layer in base_model.layers:
    print(layer.trainable)

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
Fals

In [27]:
start_epoch = 20
model_1.compile(loss = 'categorical_crossentropy', optimizer = Adam(learning_rate = 0.001), metrics = ['accuracy'])
model_1.summary()

Model: "Resnet_101"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 Data_Augmentation (Sequent  (None, None, None, 3)     0         
 ial)                                                            
                                                                 
 resnet101 (Functional)      (None, 7, 7, 2048)        42658176  
                                                                 
 global_average_pooling2d_1  (None, 2048)              0         
  (GlobalAveragePooling2D)                                       
                                                                 
 batch_normalization_5 (Bat  (None, 2048)              8192      
 chNormalization)                                                
                                                        

In [28]:
history_50 = model_1.fit(train_datagen, validation_data = (val_datagen), epochs = start_epoch+10, initial_epoch = start_epoch, verbose = 1, callbacks = [model_1chkpt, lr_scheduler], steps_per_epoch = len(train_datagen), validation_steps = len(val_datagen))

Epoch 21/30
74/74 [==============================] - ETA: 0s - loss: 1.1102 - accuracy: 0.7039
Epoch 21: val_loss did not improve from 0.70119
74/74 [==============================] - 24s 258ms/step - loss: 1.1102 - accuracy: 0.7039 - val_loss: 0.8146 - val_accuracy: 0.8566 - lr: 0.0010
Epoch 22/30
74/74 [==============================] - ETA: 0s - loss: 0.8287 - accuracy: 0.7552
Epoch 22: val_loss improved from 0.70119 to 0.42129, saving model to Trained_Models/Resnet_101
74/74 [==============================] - 30s 400ms/step - loss: 0.8287 - accuracy: 0.7552 - val_loss: 0.4213 - val_accuracy: 0.9139 - lr: 0.0010
Epoch 23/30
74/74 [==============================] - ETA: 0s - loss: 0.7217 - accuracy: 0.7916
Epoch 23: val_loss did not improve from 0.42129
74/74 [==============================] - 15s 192ms/step - loss: 0.7217 - accuracy: 0.7916 - val_loss: 0.7701 - val_accuracy: 0.6578 - lr: 0.0010
Epoch 24/30
74/74 [==============================] - ETA: 0s - loss: 0.7062 - accuracy: 0

KeyboardInterrupt: 

In [29]:
model_1.evaluate(test_datagen)

27/27 [==============================] - 2s 54ms/step - loss: nan - accuracy: 0.2462


[nan, 0.2461538463830948]

In [35]:
start_epoch = 20
for layer in base_model.layers[-50:]:
    layer.trainable = True

In [36]:
model_1.compile(loss = 'categorical_crossentropy', optimizer = Adam(learning_rate = 0.0003), metrics = ['accuracy'])
model_1.summary()

Model: "Resnet_50"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 global_average_pooling2d (  (None, 2048)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 batch_normalization (Batch  (None, 2048)              8192      
 Normalization)                                                  
                                                                 
 dense (Dense)               (None, 512)               1049088   
                                                                 
 batch_normalization_1 (Bat  (None, 512)               20

In [37]:
history_100 = model_1.fit(train_datagen, validation_data = (val_datagen), epochs = start_epoch+10, initial_epoch = start_epoch, verbose = 1, callbacks = [model_1chkpt, lr_scheduler], steps_per_epoch = len(train_datagen), validation_steps = len(val_datagen))

Epoch 21/30
47/74 [==================>...........] - ETA: 1s - loss: nan - accuracy: 0.2513

KeyboardInterrupt: 

In [30]:
start_epoch = 30
for layer in base_model.layers[-300:]:
    layer.trainable = True

In [31]:
model_1.compile(loss = 'categorical_crossentropy', optimizer = Adam(learning_rate = 0.00009), metrics = ['accuracy'])
model_1.summary()

Model: "Resnet"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 Data_Augmentation (Sequent  (None, None, None, 3)     0         
 ial)                                                            
                                                                 
 resnet152 (Functional)      (None, 7, 7, 2048)        58370944  
                                                                 
 global_average_pooling2d_1  (None, 2048)              0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dense_4 (Dense)             (None, 256)               524544    
                                                                 
 dense_5 (Dense)             (None, 128)               32896

In [32]:
history_200 = model_1.fit(train_datagen, validation_data = (val_datagen), epochs = start_epoch+10, initial_epoch = start_epoch, verbose = 1, callbacks = [model_1chkpt, lr_scheduler], steps_per_epoch = len(train_datagen), validation_steps = len(val_datagen))

Epoch 31/40
74/74 [==============================] - ETA: 0s - loss: 0.4440 - accuracy: 0.8619
Epoch 31: val_loss did not improve from 0.08031
74/74 [==============================] - 54s 291ms/step - loss: 0.4440 - accuracy: 0.8619 - val_loss: 0.1423 - val_accuracy: 0.9594 - lr: 9.0000e-05
Epoch 32/40
74/74 [==============================] - ETA: 0s - loss: 0.3214 - accuracy: 0.8937
Epoch 32: val_loss did not improve from 0.08031
74/74 [==============================] - 19s 256ms/step - loss: 0.3214 - accuracy: 0.8937 - val_loss: 0.1051 - val_accuracy: 0.9674 - lr: 9.0000e-05
Epoch 33/40
74/74 [==============================] - ETA: 0s - loss: 0.3136 - accuracy: 0.8967
Epoch 33: val_loss did not improve from 0.08031
74/74 [==============================] - 20s 266ms/step - loss: 0.3136 - accuracy: 0.8967 - val_loss: 0.2196 - val_accuracy: 0.9357 - lr: 9.0000e-05
Epoch 34/40
74/74 [==============================] - ETA: 0s - loss: 0.3187 - accuracy: 0.8920
Epoch 34: val_loss did not im

In [33]:
start_epoch = 40
for layer in base_model.layers[-450:]:
    layer.trainable = True

In [34]:
model_1.compile(loss = 'categorical_crossentropy', optimizer = Adam(learning_rate = 0.00005), metrics = ['accuracy'])
model_1.summary()

Model: "Resnet"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 Data_Augmentation (Sequent  (None, None, None, 3)     0         
 ial)                                                            
                                                                 
 resnet152 (Functional)      (None, 7, 7, 2048)        58370944  
                                                                 
 global_average_pooling2d_1  (None, 2048)              0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dense_4 (Dense)             (None, 256)               524544    
                                                                 
 dense_5 (Dense)             (None, 128)               32896

In [35]:
history_200 = model_1.fit(train_datagen, validation_data = (val_datagen), epochs = start_epoch+10, initial_epoch = start_epoch, verbose = 1, callbacks = [model_1chkpt, lr_scheduler], steps_per_epoch = len(train_datagen), validation_steps = len(val_datagen))

Epoch 41/50
74/74 [==============================] - ETA: 0s - loss: 0.3006 - accuracy: 0.9000
Epoch 41: val_loss did not improve from 0.05755
74/74 [==============================] - 84s 489ms/step - loss: 0.3006 - accuracy: 0.9000 - val_loss: 0.1595 - val_accuracy: 0.9555 - lr: 5.0000e-05
Epoch 42/50
74/74 [==============================] - ETA: 0s - loss: 0.2307 - accuracy: 0.9238
Epoch 42: val_loss did not improve from 0.05755
74/74 [==============================] - 29s 388ms/step - loss: 0.2307 - accuracy: 0.9238 - val_loss: 0.0795 - val_accuracy: 0.9743 - lr: 5.0000e-05
Epoch 43/50
74/74 [==============================] - ETA: 0s - loss: 0.2274 - accuracy: 0.9327
Epoch 43: val_loss did not improve from 0.05755
74/74 [==============================] - 27s 355ms/step - loss: 0.2274 - accuracy: 0.9327 - val_loss: 0.0609 - val_accuracy: 0.9842 - lr: 5.0000e-05
Epoch 44/50
74/74 [==============================] - ETA: 0s - loss: 0.3318 - accuracy: 0.8916
Epoch 44: val_loss did not im

In [38]:
start_epoch = 50
for layer in base_model.layers[-510:]:
    layer.trainable = True

In [39]:
model_1.compile(loss = 'categorical_crossentropy', optimizer = Adam(learning_rate = 0.000005), metrics = ['accuracy'])
model_1.summary()

Model: "Resnet"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 Data_Augmentation (Sequent  (None, None, None, 3)     0         
 ial)                                                            
                                                                 
 resnet152 (Functional)      (None, 7, 7, 2048)        58370944  
                                                                 
 global_average_pooling2d_1  (None, 2048)              0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dense_4 (Dense)             (None, 256)               524544    
                                                                 
 dense_5 (Dense)             (None, 128)               32896

In [40]:
history_500 = model_1.fit(train_datagen, validation_data = (val_datagen), epochs = start_epoch+10, initial_epoch = start_epoch, verbose = 1, callbacks = [model_1chkpt, lr_scheduler], steps_per_epoch = len(train_datagen), validation_steps = len(val_datagen))

Epoch 51/60
74/74 [==============================] - ETA: 0s - loss: 0.0817 - accuracy: 0.9788
Epoch 51: val_loss did not improve from 0.04952
74/74 [==============================] - 86s 581ms/step - loss: 0.0817 - accuracy: 0.9788 - val_loss: 0.0766 - val_accuracy: 0.9773 - lr: 5.0000e-06
Epoch 52/60
74/74 [==============================] - ETA: 0s - loss: 0.0599 - accuracy: 0.9818
Epoch 52: val_loss did not improve from 0.04952
74/74 [==============================] - 39s 521ms/step - loss: 0.0599 - accuracy: 0.9818 - val_loss: 0.1039 - val_accuracy: 0.9753 - lr: 5.0000e-06
Epoch 53/60
74/74 [==============================] - ETA: 0s - loss: 0.0651 - accuracy: 0.9818
Epoch 53: val_loss did not improve from 0.04952
74/74 [==============================] - 38s 506ms/step - loss: 0.0651 - accuracy: 0.9818 - val_loss: 0.0727 - val_accuracy: 0.9763 - lr: 5.0000e-06
Epoch 54/60
74/74 [==============================] - ETA: 0s - loss: 0.0696 - accuracy: 0.9792
Epoch 54: val_loss did not im

KeyboardInterrupt: 

In [43]:
resnet = tf.keras.models.load_model("/home/thefilthysalad/PycharmProjects/eye_detection_fundus_dataset/ML_Models/Trained_Models/Resnet")


In [46]:
resnet.evaluate(val_datagen)

32/32 [==============================] - 2s 67ms/step - loss: 0.0495 - accuracy: 0.9842


[0.04952496290206909, 0.9841740727424622]